In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 8s (32.2 kB/s)
Reading package li

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.4.0.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...|2013-01-14 00:00:00|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...| 

In [4]:
# Create & filter the vine_table
vine_df = df.filter((df.total_votes>20) & (df.helpful_votes/df.total_votes > 0.5)).select(["review_id", "star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|                Y|
|R230W87COFSTCL|          5|           21|         21|   N|     

In [5]:
vine_y_df = vine_df.filter(vine_df.vine == 'Y')
vine_y_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R6U9701C3BGO6|          3|          139|        147|   Y|                N|
|R1XK3ALB45D7N4|          5|           33|         34|   Y|                N|
|R1IZCSTLX81D6C|          5|           31|         33|   Y|                N|
|R2C8NC8EQLH4JF|          3|           45|         48|   Y|                N|
|R1JJ1YOJMOML1P|          5|           18|         21|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [6]:
vine_n_df = vine_df.filter(vine_df.vine == 'N')
vine_n_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [7]:
paid_5_star = vine_y_df.filter("star_rating == 5").count()
paid_5_star

14

In [8]:
paid_total_reviews = vine_y_df.count()
paid_total_reviews

30

In [9]:
paid_5star_percentage = paid_5_star/paid_total_reviews*100
paid_5star_percentage

46.666666666666664

In [10]:
unpaid_5_star = vine_n_df.filter("star_rating == 5").count()
unpaid_5_star

22229

In [11]:
unpaid_total_reviews = vine_n_df.count()
unpaid_total_reviews

42390

In [12]:
unpaid_5star_percentage = unpaid_5_star/unpaid_total_reviews*100
unpaid_5star_percentage

52.43925454116537